# Azure AI Agents with Model Context Protocol (MCP) Support

இந்த நோட்புக் MCP கருவிகளுடன் Azure AI Agents-ஐ பயன்படுத்தி, வெளிப்புற MCP சர்வர்களின் மேம்பட்ட திறன்களை பயன்படுத்தக்கூடிய ஒரு புத்திசாலி முகவரை உருவாக்குவது எப்படி என்பதை விளக்குகிறது.


## தேவையான NuGet தொகுப்புகளை நிறுவவும்

முதலில், Azure AI Agents Persistent தொகுப்பை நிறுவ வேண்டும், இது Azure AI Agents உடன் வேலை செய்ய தேவையான முக்கிய செயல்பாடுகளை வழங்குகிறது.


## கீ இல்லாத அங்கீகாரத்தின் நன்மைகள்

இந்த நோட்புக் **கீ இல்லாத அங்கீகாரம்** பற்றிய விளக்கத்தை வழங்குகிறது, இது பல நன்மைகளை வழங்குகிறது:
- ✅ **API கீகளை நிர்வகிக்க தேவையில்லை** - Azure அடையாள அடிப்படையிலான அங்கீகாரம் பயன்படுத்துகிறது
- ✅ **மேம்பட்ட பாதுகாப்பு** - கோடில் அல்லது கட்டமைப்பில் ரகசியங்களை சேமிக்க தேவையில்லை
- ✅ **தானியங்கும் சான்றிதழ் சுழற்சி** - Azure சான்றிதழ் வாழ்க்கைச்சுழற்சியை நிர்வகிக்கிறது
- ✅ **பாத்திர அடிப்படையிலான அணுகல்** - Azure RBAC மூலம் நுணுக்கமான அனுமதிகளை வழங்குகிறது

`DefaultAzureCredential` தானாகவே கிடைக்கக்கூடிய சிறந்த சான்றிதழ் மூலத்தை பயன்படுத்தும்:
1. மேலாண்மை அடையாளம் (Azure-ல் இயங்கும்போது)
2. Azure CLI சான்றிதழ்கள் (வளர்ச்சியின் போது)
3. Visual Studio சான்றிதழ்கள்
4. சூழல் மாறிகள் (அமைக்கப்பட்டால்)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Azure Identity தொகுப்பை நிறுவி DefaultAzureCredential பயன்படுத்தி Azure சேவைகளுடன் அங்கீகாரம் செய்யவும்.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## தேவையான பெயரிடங்களை இறக்குமதி செய்யவும்

Azure AI Agents மற்றும் Azure Identity க்கான தேவையான பெயரிடங்களை இறக்குமதி செய்யவும்.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Azure AI Agent Client (Keyless Authentication) அமைக்க

கட்டமைப்பு மாறிகள் அமைத்து **keyless authentication** மூலம் PersistentAgentsClient உருவாக்கவும்:
- **projectEndpoint**: Azure AI Foundry திட்டத்தின் முடுக்கம்
- **modelDeploymentName**: வெளியிடப்பட்ட AI மாடலின் பெயர் (GPT-4.1 nano)
- **mcpServerUrl**: MCP சர்வரின் URL (Microsoft Learn API)
- **mcpServerLabel**: MCP சர்வரை அடையாளம் காண ஒரு லேபிள்
- **DefaultAzureCredential**: நிர்வகிக்கப்பட்ட அடையாளம், Azure CLI அல்லது பிற அடையாள மூலங்களைப் பயன்படுத்துகிறது (API விசைகள் தேவையில்லை)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## MCP கருவி வரையறையை உருவாக்கவும்

Microsoft Learn MCP சர்வருடன் இணைக்க MCP கருவி வரையறையை உருவாக்கவும். இது முகவருக்கு Microsoft Learn உள்ளடக்கம் மற்றும் ஆவணங்களை அணுக அனுமதிக்கும்.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## AI முகவரியை உருவாக்கவும்

குறிப்பிட்ட மாதிரி மற்றும் MCP கருவிகளைப் பயன்படுத்தி ஒரு நிலையான AI முகவரியை உருவாக்கவும். முகவரி கீழ்க்கண்டவாறு அமைக்கப்பட்டுள்ளது:  
- GPT-4.1 நானோ மாதிரி  
- உதவிக்காக MCP கருவிகளைப் பயன்படுத்துவதற்கான வழிமுறைகள்  
- Microsoft Learn MCP சர்வருக்கு அணுகல்


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## திரைச்சூழலை உருவாக்கி செய்தி அனுப்புதல்

Azure OpenAI மற்றும் OpenAI இடையேயான வித்தியாசத்தைப் பற்றி கேள்வி கேட்கும் பயனர் செய்தியை அனுப்பி ஒரு உரையாடல் திரைச்சூழலை உருவாக்குங்கள். இது MCP கருவிகளைப் பயன்படுத்தி சரியான தகவலை வழங்குவதற்கான முகவரின் திறனை சோதிக்கும்.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## MCP கருவி வளங்களை அமைக்கவும் (Keyless)

MCP கருவி வளங்களை அமைக்கவும். உண்மையான keyless அணுகுமுறைக்கு, MCP சர்வர் Azure அடையாள அடிப்படையிலான அங்கீகாரத்தை ஆதரித்தால், தனிப்பயன் தலைப்புகளை நீக்கலாம். கீழே உள்ள உதாரணம் தேவைப்பட்டால் தலைப்புகளைச் சேர்ப்பது எப்படி என்பதை காட்டுகிறது, ஆனால் keyless சூழல்களில், இவை தேவையில்லை.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## ஏஜென்ட் இயக்கத்தை தொடங்கவும்

பயனர் செய்தியை செயலாக்க ஒரு இயக்கத்தை உருவாக்கி தொடங்கவும். ஏஜென்ட் அமைக்கப்பட்ட MCP கருவிகள் மற்றும் வளங்களைப் பயன்படுத்தி பதிலை உருவாக்கும்.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## இயக்கத்தை கண்காணித்து கருவி ஒப்புதல்களை நிர்வகிக்கவும் (Keyless)

Agent இயக்கத்தின் நிலையை கண்காணித்து, தேவையான கருவி ஒப்புதல்களை நிர்வகிக்கவும். இந்த சுழற்சி:
1. இயக்கம் முடிவடைய அல்லது நடவடிக்கை தேவைப்படும் வரை காத்திருக்கிறது
2. MCP கருவி அழைப்புகளை தானாகவே ஒப்புதலளிக்கிறது, தேவையான போது
3. Keyless அங்கீகாரம் பயன்படுத்தும்போது, MCP சர்வர் Azure அடையாளத்தை ஆதரிக்குமானால், தலைப்புகள் தேவையில்லை


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## உரையாடல் முடிவுகளை காட்டவும்

தொடரின் அனைத்து செய்திகளையும் பெறவும் மற்றும் காட்டவும், பயனர் கேள்வியும் முகவரின் பதிலும் சேர்த்து. செய்திகள் கால வரிசைப்படி நேரக்குறிப்புகள் மற்றும் பாத்திர சுட்டிகளுடன் காட்டப்படும்.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையைப் பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. நாங்கள் துல்லியத்திற்காக முயற்சிக்கின்றோம், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது தவறான தகவல்கள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளுங்கள். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
